# Portfolio Optimization in Python 101 - PyPortfolioOpt edition

## Setup

In [1]:
import requests
import pandas as pd

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov

# api key
from api_keys import FMP_API_KEY

## Downloading data

In [2]:
FAANG_TICKERS = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]
START_DATE = "2023-01-01"

In [3]:
def get_adj_close_price(symbol, start_date):
    hist_price_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&apikey={FMP_API_KEY}"
    r_json = requests.get(hist_price_url).json()
    df = pd.DataFrame(r_json["historical"]).set_index("date").sort_index()
    df.index = pd.to_datetime(df.index)
    return df[["adjClose"]].rename(columns={"adjClose": symbol})

In [4]:
price_df_list = []
for ticker in FAANG_TICKERS:
    price_df_list.append(get_adj_close_price(ticker, START_DATE))
prices_df = price_df_list[0].join(price_df_list[1:])
prices_df = prices_df[:"2024-05-02"]
prices_df

,META,AAPL,AMZN,NFLX,GOOGL
date,,,,,
2023-01-03,124.61,124.22,85.82,294.950012,89.12
2023-01-04,127.24,125.50,85.14,309.410004,88.08
2023-01-05,126.81,124.17,83.12,309.700012,86.20
2023-01-06,129.88,128.74,86.08,315.549988,87.34
2023-01-09,129.33,129.26,87.36,315.170013,88.02
...,...,...,...,...,...
2024-04-26,443.29,169.30,179.62,561.230000,171.95
2024-04-29,432.62,173.50,180.96,559.490000,166.15
2024-04-30,430.17,170.33,175.00,550.640000,162.78


In [5]:
returns_df = prices_df.pct_change().dropna()
returns_df

,META,AAPL,AMZN,NFLX,GOOGL
date,,,,,
2023-01-04,0.021106,0.010304,-0.007924,0.049025,-0.011670
2023-01-05,-0.003379,-0.010598,-0.023726,0.000937,-0.021344
2023-01-06,0.024209,0.036804,0.035611,0.018889,0.013225
2023-01-09,-0.004235,0.004039,0.014870,-0.001204,0.007786
2023-01-10,0.027217,0.004487,0.028732,0.039249,0.004544
...,...,...,...,...,...
2024-04-26,0.004327,-0.003473,0.034260,-0.006321,0.102244
2024-04-29,-0.024070,0.024808,0.007460,-0.003100,-0.033731
2024-04-30,-0.005663,-0.018271,-0.032935,-0.015818,-0.020283


## Portfolio Optimization

In [11]:
# Calculate the annualized expected returns and the covariance matrix
avg_returns = mean_historical_return(prices_df, compounding=False)
cov_mat = sample_cov(prices_df)

In [12]:
# Find the minimum volatility portfolio using the Efficient Frontier
ef = EfficientFrontier(avg_returns, cov_mat)  
min_vol_portf_weights = ef.min_volatility()
min_vol_portf_weights

OrderedDict([('META', 0.0),
             ('AAPL', 0.7398377197342494),
             ('AMZN', 0.1066118207587511),
             ('NFLX', 0.0715698232942175),
             ('GOOGL', 0.0819806362127821)])

In [13]:
ef.portfolio_performance(verbose = True, risk_free_rate=0);

Expected annual return: 35.0%
Annual volatility: 19.6%
Sharpe Ratio: 1.78


In [14]:
# Find the max Sharpe portfolio using the Efficient Frontier
ef = EfficientFrontier(avg_returns, cov_mat)  
max_sharpe_portf_weights = ef.max_sharpe(risk_free_rate=0)
max_sharpe_portf_weights

OrderedDict([('META', 0.5046490050802513),
             ('AAPL', 0.0),
             ('AMZN', 0.169952289630033),
             ('NFLX', 0.2113174241483321),
             ('GOOGL', 0.1140812811413838)])

In [15]:
ef.portfolio_performance(verbose = True, risk_free_rate=0);

Expected annual return: 80.9%
Annual volatility: 30.6%
Sharpe Ratio: 2.64
